In [1]:
import numpy as np
import matplotlib.pylab as plt
import os

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else "cpu"

### Generating Synthetic Dataset
#### This will reflect how well the network learns indexing.

In [3]:
# generate N vectors of 600 bytes
# learn indexing into second byte
def gen_data(N=5000, d=32, low=0, high=127, target_idx=1):
    data = np.random.randint(low=low, high=high, size=(N,d))
    return data, data[:, target_idx]

In [159]:
data = gen_data()
#print(data)
features = data[0]
print(features[0:3])
labels = data[1]
print(labels[0:3])
test_data = gen_data(N=500)
features_test = test_data[0]
print(features_test[0:3])
labels_test = test_data[1]
print(labels_test[0:3])

[[  2 105  31  45  78 121  48  82  57 106  16   6  23  90  88  63  58  97
   25  64  15  39 107  83  20  45   6 101  43   6  11  10]
 [ 81   2  55  78 109   7  73  19   9 124  22  88  76 113  89  31  68   9
    2  61 110 124 124  83  34  89  29  69 116  25  82 108]
 [ 87  92  61  44  55  62  99  28 119  80  25 122 102  38 124  49  54  81
   50  85  27  81  86  30  25  53   6  65   0  91  76 119]]
[105   2  92]
[[ 15   8  45  17  21 110 113  12  79  33  76  82  94  60 108  54 122 122
   22 105  69  23  35  63  97  12  90  86  29  88  11 109]
 [ 61  46  13   4   7   8  67  67  33 101  93  89  17  11  49  68   9  70
   13 114  78   8   3  50  83 125  75  45  10 103  67  97]
 [ 38   0  77 108  18  47  75  69  77  66 124  49  11  45 112  68  10  73
  110 111  39  60  79  66  65 119  42  39 115  74  29  88]]
[ 8 46  0]


### Defining Conv. NN

In [160]:
net = nn.Sequential(nn.Conv2d(1, 8, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(8, 16, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(16, 32, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Flatten(start_dim=1, end_dim=-1),
                   )

In [161]:
net(torch.from_numpy(features[0]).unsqueeze(0).unsqueeze(0).unsqueeze(0).float()).shape 

torch.Size([1, 832])

In [175]:
net = nn.Sequential(nn.Conv2d(1, 8, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(8, 16, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(16, 32, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Flatten(start_dim=1, end_dim=-1),
                    nn.Linear(832, 127)
                   )

In [176]:
# error between real and predicted 2 16-bit values
criterion = nn.CrossEntropyLoss()
#criterion = nn.MSELoss()

In [177]:
optimizer = optim.Adam(net.parameters(), lr=1e-4)

### Training and Validation

In [178]:
def validate(net, test_data, test_target, net_type=None):
    net = net.eval()
    with torch.no_grad():
        test_data_torch = torch.from_numpy(test_data).float().to(device)
        test_target_torch = torch.from_numpy(test_target).float().to(device)

        if net_type=='conv':
            test_data_torch = test_data_torch.unsqueeze(1).unsqueeze(2)

        test_pred = net(test_data_torch).argmax(dim=1)
        test_accuracy = (test_target_torch == test_pred).float().mean().to('cpu').detach().numpy()

        #test_pred = np.rint(net(test_data_torch).detach().numpy())
        #test_pred = net(test_data_torch).detach().numpy()
        #test_accuracy = (test_target_torch.detach().numpy() == test_pred).mean()

        test_rmse = (np.sqrt((test_pred-test_target_torch.detach().numpy())**2).mean())

    return test_accuracy, test_rmse

def train_net(net, train_data, train_target, test_data, test_target, N_epochs=10, print_freq=100, net_type=None):
    net = net.to(device)
    train_data_torch = torch.from_numpy(train_data).float().to(device)
    train_target_torch = torch.from_numpy(train_target).to(device)
    
    if net_type=='conv':
        train_data_torch = train_data_torch.unsqueeze(1).unsqueeze(2)

    net = net.train()
    for i in range(N_epochs):
        #pred = net(train_data_torch.float())
        #loss = criterion(pred, train_target_torch.float())
        pred = net(train_data_torch)
        loss = criterion(pred, train_target_torch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % print_freq == 0:
            train_accuracy, train_rmse = validate(net, train_data, train_target, net_type=net_type)
            test_accuracy, test_rmse = validate(net, test_data, test_target, net_type=net_type)
            net = net.train()
            #pred = pred.detach().numpy()
            #print("pred: ", pred[i], "target: ", train_target[i])
            print(f'Epoch = {i} loss={loss} train_acc = {train_accuracy:.3f} test_acc = {test_accuracy:.3f} train_rmse = {train_rmse:.3f}  test_rmse = {test_rmse:.3f}')
            
    return net

In [179]:
net = train_net(net, features, labels, features_test, labels_test, N_epochs=1000, print_freq=10, net_type='conv')

torch.save(net, "conv_0")

Epoch = 0 loss=5.689403533935547 train_acc = 0.007 test_acc = 0.004 train_rmse = 37.460  test_rmse = 36.148
Epoch = 10 loss=5.132148742675781 train_acc = 0.011 test_acc = 0.010 train_rmse = 39.927  test_rmse = 39.478
Epoch = 20 loss=4.987150192260742 train_acc = 0.012 test_acc = 0.012 train_rmse = 43.177  test_rmse = 42.106
Epoch = 30 loss=4.89721155166626 train_acc = 0.018 test_acc = 0.016 train_rmse = 41.873  test_rmse = 42.172
Epoch = 40 loss=4.81852912902832 train_acc = 0.019 test_acc = 0.016 train_rmse = 40.435  test_rmse = 40.908
Epoch = 50 loss=4.74665641784668 train_acc = 0.024 test_acc = 0.012 train_rmse = 39.693  test_rmse = 40.590
Epoch = 60 loss=4.67628812789917 train_acc = 0.033 test_acc = 0.010 train_rmse = 38.809  test_rmse = 40.546
Epoch = 70 loss=4.605659484863281 train_acc = 0.042 test_acc = 0.010 train_rmse = 37.377  test_rmse = 39.946
Epoch = 80 loss=4.533350944519043 train_acc = 0.053 test_acc = 0.010 train_rmse = 35.833  test_rmse = 38.678
Epoch = 90 loss=4.459099

KeyboardInterrupt: 